# Lab 3: Fewshot ICL

As knowledge graph requires background in SPARQL and/or LLM finetuning, this lab won't be totally related to what you saw in today's course.

We'll be delving into In Context Learning (ICL), in particular ICL fewshot, and trying to understand how it works and when to use it. To do this, we'll be using the Transformer Library, a Mistral LLM and an emotion classification dataset.


The laboratory is divided into 4 sections:
0. Setup: This section is dedicated to installing modules, loading models and loading data.You don't need to code, just run it.
1. Zeroshot Classification: Some of you may have had trouble finding a prompt that always returned a “well-formed” answer in the last lab. In this section, we'll use a “well-formed” prompt to perform zeroshot classification.
2. Fewshot Classification - Random Retrieval: One of the most common methods of improving ICL classification is to add demonstrations to the prompt. This helps the LLM to “properly format” the response and can also give semantic information about how to solve the task. In this section, we will use random retrieved demonstration and compare the results with those of section 1.
3. Fewshot Classification - Vector-based Retrieval: Extracting random demonstrations in fewshot classification can introduce bias. In addition, most semantically relevant demonstrations are not taken into account. As with did with RAG, we will use a vector representation of the example to retrieve the most relevant demonstrations.
4. Constrained Decoding: Finally, we'll discovering the `outlines` library, which contains modules that are useful to do constrained decoding.

At the end of each section (except section 0.), there's a question to answer.

## 0. Setup

In [ ]:
!pip install transformers bitsandbytes accelerate datasets outlines scikit-learn

In [ ]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import torch

# Put your hugging face token here: https://huggingface.co/docs/hub/en/security-tokens
# You need to fill the access form with your huggingface account on this link: https://huggingface.co/mistralai/Ministral-8B-Instruct-2410
hf_token = ""
llm_name = "mistralai/Ministral-8B-Instruct-2410"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left", token=hf_token)
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
    token=hf_token
)
# Set LLM on eval mode.
llm.eval()


In [ ]:
# Set up our generation configuration.
# We set max_new_token to 128 to reduce computation time (we may also lose some accuracy).
# We disable beamsearch to ensure reproducibility (we may lose some accuracy).
generation_config = GenerationConfig(
  max_new_tokens = 128,
  do_sample=False,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [ ]:
from datasets import load_dataset
import random
random.seed(42)

id2label = {0:"sadness", 1:"joy", 2:"love", 3:"anger", 4:"fear", 5:"surprise"}


# Dataset: https://huggingface.co/datasets/dair-ai/emotion
ds = load_dataset("dair-ai/emotion", "split")
examples = [{"text":ex["text"], "label":id2label[ex["label"]]}for ex in ds['test'].to_list()]
random.shuffle(examples)

# Split examples and keep only a few samples to have short computation time.
test, train = examples[:100], examples[100:500]
print(f"Train len {len(train)}. Test len {len(test)}")
print(f"First example of test:\n{test[0]}")

## 1. Zero-shot Classification

It's very similar to what you've done last time, so we're providing you with most of the code. The only thing you need to code yourself is the parse_answer function.
- We adapted the recommended classification prompt from: https://docs.mistral.ai/guides/prompting_capabilities/
- The purpose of this function is to return the first occurrence of a correct label (sadness, joy, love, anger, fear, surprise)
- We want to return "" if no answer is found.
- You can use regex or string functions.

There is a cell below to test your code. The output should be:

```
##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the category. Do not include the word "Category". Do not provide explanations or notes.

<<<
Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label:
>>>
# sadness
# sadness
```

In [ ]:
import re

zeroshot_prompt = """
You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

<<<
Sentence: {sentence}
Label:
>>>
""".strip()


def generate(prompt, llm=llm, generation_config=generation_config):

  # Create turns with the given prompt
  turns = [
    {'role':'user', 'content':prompt}
  ]

  # Tokenize turns.
  input_ids = tokenizer.apply_chat_template(turns, return_tensors='pt').to('cuda')

  # Ensure we don't use gradient to save memory space and computation time.
  with torch.no_grad():
    outputs = llm.generate(
      input_ids,
      generation_config
    )

  # Recover and decode answer.
  answer_tokens = outputs[0, input_ids.shape[1]:-1]
  return tokenizer.decode(answer_tokens).strip()


def parse_answer(answer):
  # TODO

In [ ]:
# Test your code

example = test[0]

prompt = zeroshot_prompt.format(sentence=example["text"])
answer = generate(prompt)
prediction = parse_answer(answer)

print("##### Example 0 #####")
print(f"# {prompt}")
print(f"# {answer}")
print(f"# {prediction}")

Now apply the fewshot prompt on the full test dataset. You need report:
- Accuracy (recall: number of correct answers divided by number of samples)
- Ratio of missing answer (i.e "." answer)

It should take 3 to 5 minutes to run.

In [ ]:
from tqdm import tqdm

for example in tqdm(test): # tqdm allow you to track the progression of your loop.
  # TODO


Note: We always find an answer, because we've used a “well-formed” prompt and because Mistral is good at following this type of instruction. If you try with the Lama-3, some answers may be missing.

**Question: Are we sure that all these answer are "well-formed" answer ?**

## 2. Fewshot Classification - Random Retrieval:

Now we have a working zeroshot solution. Our next next step is to use demonstrations. We will start be implementing a random few shot generation. You need to implement 3 functions:

- format_demo, wich format a given example into a demonstration string
- format_demos, wich format a given list of example into a demonstration string (try to use format_demo)
- get_random_demo, wich return k random examples. (you should use random.choice. https://docs.python.org/3/library/random.html)


There is a cell below to test your code. The output should be:
```
##### format_demo #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise.


##### format_demos #####
# Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label: surprise.

Sentence: im feeling optimistic to finish out these last two weeks strong and probably continue with what i have been doing
Label: joy.

Sentence: i feel complacent and satisfied
Label: joy.

Sentence: im the only one with all the feelings and emotions and thats just pathetic of me to do so
Label: sadness.

Sentence: i just sat there in my group feeling really depressed because my book just had to go missing at this time
Label: sadness.


##### Example 0 #####
# You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

####
Here are some examples:

Sentence: i feel inspired so many thing i want to write down
Label: joy.

Sentence: i feel like i should have some sort of rockstar razzle dazzle lifestyle but i would at least like to spend a third of my life doing something i feel is worthwhile
Label: joy.

Sentence: i continue to write this i feel more and more distraught
Label: fear.

Sentence: i feel that third situation pretty much sums up my feelings toward this title
Label: joy.

Sentence: i remember wanting to fit in so bad and feeling like no one liked me
Label: love.
####

<<<
Sentence: i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
Label:
>>>
# sadness
# sadness
```

In [ ]:
fewshot_prompt = """
You're an expert in sentiment analysis. Your task is to classify the sentence emotion after <<<>>> with one of the following predefined labels:

sadness
joy
love
anger
fear
surprise

You will only respond with the label. Do not include the word "Label". Do not provide explanations or notes.

####
Here are some examples:

{examples}
####

<<<
Sentence: {sentence}
Label:
>>>
""".strip()

def format_demo(demo):
  # TODO

def format_demos(demos):
  # TODO

def get_random_demo(k, train=train):
  # TODO

In [ ]:
# Test your code !


print("##### format_demo #####")
print(f"# {format_demo(test[0])}")


print("\n\n##### format_demos #####")
print(f"# {format_demos(test[:5])}")


random.seed(42)

example = test[0]
demos = format_demos(get_random_demo(5))

prompt = fewshot_prompt.format(examples=demos, sentence=example["text"])
answer = generate(prompt)
prediction = parse_answer(answer)

print("\n\n##### Example 0 #####")

print(f"# {prompt}")
print(f"# {answer}")
print(f"# {prediction}")


Now apply the fewshot prompt on the full test dataset. You need report:
- Accuracy (recall: number of correct answers divided by number of samples)
- Report them for k=1 and k=5

It should take 5 to 7 minutes to run.



In [ ]:
from tqdm import tqdm

random.seed(42)

for example in tqdm(test): # tqdm allow you to track the progression of your loop.
  # TODO

**Question: What are the limits of using a single demonstration? What are the limits of using too many demonstrations?**

## 3. Fewshot Classification - Vector-based Retrieval

Now, we want to improve demonstrayion by the vector representation of our sentence. This is close to what we did when we used RAG on wikipedia page. But here, we'll do it manually and step by step.

To do so, we need to calculate the vector representation of our training dataset. To do this, we'll code a function that returns a vector for a given example. We'll use our LLM hidden states to do this. It's not optimal, but we won't have to load another model.

First, look at the mistral architecture:

```
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=131072, bias=False)
)
```
There are 36 transformer layers and 1 language model (LM) layer. Each layer will take the following shape: [1, N_TOKENS, N_PARAMS]. We want to extract the vector of the last token from the last transformer. To do so:
- Encode the sentence without any template. `tokenizer.encode(...)`
- Use the `output_hidden_states` keyword of the llm forward function.
- Select the last transformer layer (be careful, don't take the LM layer).
- Select the last token.
- Convert the vector to numpy `.to('cpu').float().numpy()` and return it.

There is a cell below to test your code. The output should be:
```
# (4096,)
# [ 4.59375    -9.          0.80078125 ...  0.890625   -0.20019531
 -0.62109375]
```

In [ ]:
def get_hidden_repr(text, llm=llm, prompt_template=zeroshot_prompt):
  # TODO


In [ ]:
example = train[0]
vector = get_hidden_repr(example["text"])
print("#", vector.shape)
print("#", vector)

Now, we need to get the hidden represation vector for all examples in the train and the test datasets.

You should store the vector directly in the example dict: `example["vector"] = ...`

Both should take 3 - 5 mins to run.

In [ ]:
for example in tqdm(train): # tqdm allow you to track the progression of your loop.
  # TODO

In [ ]:
# Same for test examples !
for example in tqdm(test): # tqdm allow you to track the progression of your loop.
  # TODO

Now that we have our vector representations. We want a function that compute the cosine similarity between 2 examples.

- Use the function from sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
- Be careful, you have to reshape each vector to: [1, 4096]

There is a cell below to test your code. The output should be:
```
# a . a = 1.0000019073486328
# a . b = 0.930396318435669
```

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(example_a, example_b):
  # TODO

In [ ]:
# Test your code !

a, b = train[0], train[1]

print(f"# a . a = {compute_similarity(a, a)}")
print(f"# a . b = {compute_similarity(a, b)}")

Last step, we want a function that retrieve the k more similar demonstrations of the train examples given a test example.

There is a cell below to test your code. The output should be:
```
# surprise - i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history
#  joy - i feel lucky that theyve chosen to share their lives with me

joy - i feel our world then was a much more innocent place

joy - i know he does the same thing for so many passersby i feel special truly welcome in his country

joy - i do know that i tell some people if i feel that their question is sincere some of my sacred treasures

anger - i feel appalled that i took advantage of my old friend s kindness

```

In [ ]:
def get_k_similar_demo(example, k, train=train):
  # TODO

In [ ]:
# Test your code !
example = test[0]
print(f"# {example['label']} - {example['text']}")
print("# ", "\n\n".join([f"{ex['label']} - {ex['text']}" for ex in get_k_similar_demo(example, 5)]))

Now apply the fewshot prompt on the full test dataset. You need report:
- Accuracy (recall: number of correct answers divided by number of samples)
- Report them for k=1 and k=5

It should take 5 to 7 minutes to run.

Your results should be:
```
##### k=1 #####
Accuracy:  0.65
##### k=5 #####
Accuracy:  0.63
```

In [ ]:
from tqdm import tqdm

for example in tqdm(test): # tqdm allow you to track the progression of your loop.
  # TODO

**Question: What could be the main issue with this approach? How can it be mitigated?**

## 4. Constrained Decoding

Last exercise, we will use the `outlines` package to do constrained generation. This main idea is to guide the generation of the LLM to get the good output formats.

We will use the choices module. Here is the documentation: https://dottxt-ai.github.io/outlines/latest/reference/generation/choices/

There is an example below on how to use it on 1 example. We let you apply this methods to the test dataset. You need report:
- Accuracy (recall: number of correct answers divided by number of samples)
- Ratio of missing answer (i.e "E." answer)
- Report them for k=1 and k=5

It should take 3 to 5 minutes to run.

Your results should be:
```
Accuracy:  0.38
```

In [ ]:
from outlines import models, generate

# TODO

**Question: Now that you've used all these solutions, when should you use zeroshot? when should you use fewshot? when should you use constrained decoding?**

## Bonus

Try to use differents modules of `outlines` like json, pydantic or regex ...

Compare this results with previous ones !